# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons.
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [5]:
import CreateSamples
import OT2Commands as ALH
import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api

# Would not load
import importlib # for reloading packages
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
importlib.reload(CreateSamples)
importlib.reload(ALH)
importlib.reload(ographing)

<module 'OT2Graphing' from 'C:\\Users\\Edwin\\Desktop\\Repos\\OT2Protocols\\ot2protocol\\Ouzo_OT2_Sampling\\OT2Graphing.py'>

## Step 1: Set up the experiment dictionary.
* The first step to planning an experiment is to load the experiment variables and inputs from a csv file. Every variable should have an input with an acceptable datatype. At the moment this step is done by opening a CSV file in Excel, where the first column is the name of the variable and the adjacent column is the variable value. The default delimitter is (,). 
    * Reading directly as csv is fine but it requires you have all data values in a string, so we switch to load from a csv using pandas, which will load everything from a csv as within a string node, then we can use ast.literal_eval to unpack this and the appropiate datatypes. This prevent you from having to put quotation marks around each variable value when planning the experiment.
    * Loading from excel can be done in a similar mannis avoided due not having xlrd or openpyxl depdenency native to python, and the opentrons being limited in the packages we can add/update. Hence we default to a CSV.

In [58]:
import csv

In [70]:
path = r"C:\Users\Edwin\Desktop\Repos\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Testing Plans\02_17_20_Reproducibility.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path) 

ValueError: malformed node or string: <ast.Name object at 0x00000200A620DD00>

In [82]:
i = pd.read_csv(path)
i['Value'][4]

'"path"'

In [44]:
import ast

In [55]:
ex_path = r"C:\Users\Edwin\Desktop\Repos\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Testing Plans\Excel_test.xlsx"
a = pd.read_excel(ex_path)
variable_names = a['Variable Name']
values = a['Value']
i = ast.literal_eval(values[12])
type(values[0])

int

In [32]:
df = pd.Series(experiment_csv_dict).rename_axis('Variable Name').reset_index(name='Variable Value')
df

,Variable Name,Variable Value
0,Variable Name,Value
1,Protocol Version,2.0
2,Experimenter,Edwin Antonio
3,Project Tag,Ethanol and PFH Scan
4,Experiment Tag,Experiment 1 Trial 1
5,Chemical Database Path,path
6,Sample Amount,0.8
7,Sample Unit,g
8,Component Good Solvent Index (Only Ouzo),4
9,Component Poor Solvent Index (Only Ouzo),5


In [3]:
# Step 1: Call and Collect

labware_dir_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

# these variables although have the same names when searching for the stocks should now replace those for the actual selectede one. 
path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Testing Plans\02_17_20_Reproducibility.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path) # explain specfically what this is !

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Edwin\\Desktop\\OT2Protocols\\ot2protocol\\Ouzo_OT2_Sampling\\Custom Labware'

In [ ]:
# Step 2: Create wtf and vol dataframes, combine into one as to make any filtering affect both.
wtf_sample_canidates = CreateSamples.generate_candidate_lattice_concentrations(experiment_csv_dict)
volume_sample_canidates = CreateSamples.calculate_ouzo_volumes_from_wtf(wtf_sample_canidates, experiment_csv_dict)
complete_df = CreateSamples.combine_df(wtf_sample_canidates, volume_sample_canidates) # unfiltered
complete_df = pd.concat([complete_df]*48, ignore_index=True)

In [ ]:
# Step 3: Apply filters through df based logic, currently 4 filters exist (volume, total, general min and general max)

# First filter for pipette volume constraints, optional Volume Restriction to select certain components for filter application ("stock" must be in column name)
complete_df_f1 = CreateSamples.pipette_volume_restriction_df(complete_df, 30, 1000, experiment_csv_dict['Volume Restriction']) # last argument is optional

# Second filter for overall total volume_restriction, call max destination well volume ("Total Sample Volume" must be in column name)
# max_dest_well_volume = ALH.find_max_dest_volume_labware(experiment_csv_dict, custom_labware_dict)
complete_df_f2 = CreateSamples.total_volume_restriction_df(complete_df_f1,1400)

#Thrid filter for any general max or min filtering you would like
final_complete_df = complete_df_f2#CreateSamples.general_max_restriction(complete_df_f2, 360, 'pfh-ethanol-stock uL')

# finalize it
final_wtf_df = CreateSamples.isolate_common_column(final_complete_df, 'wtf')
final_volume_df = CreateSamples.isolate_common_column(final_complete_df, 'stock')
final_wtf_df

In [ ]:
# ographing.xy_scatter_df_compare(complete_df, final_complete_df, 'ethanol molf', 'pfh molf')
# ographing.xy_scatter_df(final_complete_df, 'ethanol wtf', 'pfh molf')

In [ ]:
chem_database_path = r"C:\Users\Edwin\Desktop\OT2Protocols\ot2protocol\Ouzo_OT2_Sampling\Chemical Database.xlsx"
stock_prep_df = CreateSamples.calculate_stock_prep_df(experiment_csv_dict, final_volume_df, chem_database_path)
# pd.set_option('display.float_format', lambda x: '%.2e' % x)
stock_prep_df

In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
max_vol = 20000
stock_ranges = ALH.stock_well_ranges(final_volume_df, max_vol) # set up volumes orders
stock_ranges

In [ ]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, experiment_csv_dict) # the protocol above has been modified globally!
info = ALH.pipette_stock_volumes(protocol, loaded_dict, final_volume_df, stock_ranges)

In [ ]:
CreateSamples.create_csv(r"C:\Users\Edwin\Desktop\test", info['info concat'], final_wtf_df.values, experiment_csv_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\test")
